# Graph-Optimizer beta testing round 2

## Tool description
In short, the Graph-Optimizer tool performs the following functions:
- Predicts the execution time (in milliseconds) and energy consumption (in Joules) for a given BGO or DAG of BGOs on a specific hardware configuration.
- Returns the model in symbolical form with graph properties as symbols or predicts execution times if the graph properties are specified.
- This is done via an API where issuing a POST request to `<api_url>/models` with the BGO DAG and hardware configuration returns an annotated DAG with calibrated symbolical models. Calling `<api_url>/evaluate` with the BGO DAG, hardware configuration, and graph properties returns an annotated DAG with predicted execution times.


### What is a BGO?
A BGO, or Basic Graph Operation, is an atomic graph operation, that can serve as a building block for constructing larger graph _workloads_.
A single BGO can have multiple implementations, possibly targeting different hardware platforms (e.g., CPU or GPU).
A workload is a Directed Acyclic Graph (DAG) of BGOs, where the nodes are BGOs and the edges represent data dependencies between them.
An example of such a DAG is shown below:

<img style="margin-bottom: -245px" src="dag.svg">

In this workflow, we start with the Betweenness Centrality (BC) BGO, followed by the Breadth First Search (BFS) and Find Max BGO's. Finally, we have the Find Path BGO to conclude. This example is a workload that outputs the path from the root node to the node with the highest betweenness centrality, which is the most popular node in the graph.

#### BGO implementations

Each BGO can have multiple implementations, on different hardware devices. These different implementations can have varying performance based on the system. The task of the optimizer is to identify which BGO implementation is optimal for a given system. In this tutorial, we have various CPU and GPU implementations for the BGO's.

### Performance modeling

#### Analytical modeling
An implementation of a BGO can have a symbolic model that describes its execution time and energy consumption as functions of graph properties and hardware characteristics. Specifically, these hardware characteristics refer to the execution times of atomic operations in hardware, or operations considered to be atomic, such as reading a value from memory, writing a value to memory, performing an integer addition, and so on. These characteristics are obtained through _microbenchmarks_ run on the hardware where the BGO will be executed. The values obtained from the microbenchmarks are used to calibrate the symbolic models, which then provide the execution time and energy consumption of the BGO based solely on the graph properties. This approach allows for predicting the execution time and energy consumption of a BGO on a specific hardware configuration without needing to run the BGO on the hardware for any particular input graph.

Such a calibrated model might look like this:

$T_{BGO} = 561n \times 924m + 91n^2$,

where $n$ is the number of nodes in a graph, $m$ is the number of edges in the graph, and $T_{BGO}$ is the execution time of the BGO in milliseconds.

#### Sampling based prediction
Analytical models can provide valuable insights into performance, but are difficult and time consuming to make accurate. Therefore, a second prediction approach is implemented. By sampling the graph to a fraction of its full size, and running a BGO on this sampled version, we can observe the execution time of the BGO on the sampled graph, and multiply this by some scalar, which depends on the size of the graph and sampling rate, to get a prediction for the execution time of the BGO on the full graph. A big drawback of sampling based prediction is the prediction overhead. Whereas analytical modeling gives instant predictions, sampling based prediction does give a significant overhead.

In [ ]:
# Includes for the entire notebook
import multiprocessing
import requests
import subprocess
import time
import json
import os

os.chdir("/workspace")

import networkx as nx
import scipy as sp

from datetime import datetime

from ipywidgets import interact, dlink, Dropdown, FloatSlider, ToggleButtons
from ipyfilechooser import FileChooser
from IPython.display import Markdown, display, clear_output

from beta_testing.plot_predictions import plot_annotated_dag, analytical_prediction_validation, sampling_prediction_validation
from benchmarks.microbenchmarks import all_benchmarks

## Steps to use the Graph-Optimizer tool
### Step 1: Specify input DAG of BGO's

The first step in using Graph Optimizer is to specify the input DAG of BGO's. From the above list, select the BGO's you want to use and specify the input DAG.
This DAG should include one or multiple BGOs and their dependencies. The BGO name should match the name of the BGO folder in the `models` directory.
There are currently 7 BGO's available. These are:
- bc (Betweenness Centrality)
- pr (PageRank)
- bfs (Breadth First Search)
- find_max (Find Max)
- find_path (Find Path)
- cc (Connected Components)
- tc (Triangle Counting)

The dependencies should be specified as a list of BGO id's that the current BGO depends on. For instance, consider the following example with multiple BGOs and dependencies, representing the DAG shown in the introduction:

In [ ]:
input_dag = [
    {
        "id": 0,
        "name": "pr",
        "dependencies": []
    },
    {
        "id": 1,
        "name": "find_max",
        "dependencies": [0]
    },
    {
        "id": 2,
        "name": "bfs",
        "dependencies": [0]
    },
    {
        "id": 3,
        "name": "find_path",
        "dependencies": [1,2]
    }
]

We recommend using this configuration for this tutorial, but feel free to play around with the different BGO's after completing.

### Step 2: Specify hardware configuration
The hardware configuration aims to describe all available hardware components in a system or data center. The hardware information is used for the calibration of the performance models.

To specify a custom hardware configuration, you need to provide the configuration in JSON format. This configuration should list all unique available hosts in the data center, including details about CPUs and, if applicable, GPUs. Running microbenchmarks is part of this process and is done automatically with a script. An example configuration is provided below.

In [ ]:
hardware = {
    "hosts": [
        {
            "id": 1,
            "name": "H01",
            "cpus": {
                "id": 1,
                "name": "intel xeon",
                "clock_speed": 2.10,
                "cores": 16,
                "threads": 32,
                "wattage": 35,
                "amount": 2,
                "benchmarks": {
                    "T_int_add": 1.739769,
                    "T_float_add": 0.4340147,
                    "T_q_push": 7.381785,
                    "T_heap_insert_max": 37.04162,
                    "T_push_back": 5.958142,
                    "T_DRAM_read": 66.34084759860137
                }
            }
        }
    ]
}

#### Obtaining hardware information

The following code cell will fetch the hardware information for the system that this notebook is running on, and fill it in in the hardware configuration JSON object.

***Note:*** depending on your system, the following cell might give an error. If this is the case, please fill in the hardware information manually to the best of your knowledge.

In [ ]:
lscpu = {k.strip(): v.strip() for item in subprocess.check_output(['lscpu']).decode('utf-8').split('\n') for k, _, v in [item.partition(':')]}
hardware['hosts'][0]['cpus']['name'] = lscpu['Model name']
hardware['hosts'][0]['cpus']['cores'] = int(lscpu['Core(s) per socket'])
hardware['hosts'][0]['cpus']['threads'] = int(lscpu['Thread(s) per core']) * hardware['hosts'][0]['cpus']['cores']
hardware['hosts'][0]['cpus']['amount'] = int(lscpu['Socket(s)'])

#### Automated microbenchmarks

Running the following python cell will run the microbenchmarks on your machine (this should take a couple of seconds, probably no longer than a minute), and insert them into the hardware configuration.

The resulting values are the measured values for each operation in nanoseconds.

In [ ]:
# Run microbenchmarks on this machine
local_benchmarks = all_benchmarks()

# Pretty print the benchmarks
print(json.dumps(local_benchmarks, indent=4))

# Assign obtained values to the hardware description
hardware["hosts"][0]["cpus"]["benchmarks"] = local_benchmarks

### Step 3: Run the prediction server

The next step is running the prediction server, and getting the performance models for the input DAG for your specific hardware configurations.
This step can be divided into two subtasks. First we start the prediction server, and then we submit a POST request to the server to get the performance models.

1. Start the server using flask, by running the following command from the root directory:
    ```bash
    flask --app api/api.py run
    ```
    _(If you are using windows, use `python -m flask --app api/api.py run`)_

    This will start the server on `localhost:5000`. Use the following command to start the server on a different port:
    ```bash
    flask --app api/api.py run --port <port_number>
    ```
    You can either do this step, or run the python cell below, which will start the server for you.
    - **Note**: If you run the server via the cell below, make sure to wait for the server to start before proceeding to the next cells.
    - If you are finished with the experiments, you can stop the server by running the last cell in this document, which will terminate the server.
2. Run the prediction by submitting a POST request to the api
    - For obtaining the calibrated symbolical models, issue a post request to `localhost:<port_number>/models`, with the following post data:
        - "input_dag": The input BGO DAG in JSON format, as a string.
        - "hardware": The hardware configuration in JSON format, as a string.

***Note:*** If at any point later in this tutorial the jupyter kernel is interrupted, the server will stop. It needs to be manually activated by running this cell again.



In [ ]:
# Start the server
port = 5000
url = f'http://localhost:{port}/'

def run_server():
    !flask --app api/api.py run --port {port}

server = multiprocessing.Process(target=run_server)
server.start()

#### Interpreting the results

When submitting the post request to the API, the response will be the input DAG, but annotated with the calibrated symbolical models. The models will be in the form of a string representing a mathematical formula, with the graph properties as parameters.

For demonstration purposes, a dropdown and slider are provided below, which allow you to change the microbenchmarking parameters and see the impact they have on the performance models.

#### Submit a request to the prediction server by executing the cells below, and observe how the models change when altering the microbenchmarking parameters.

In [ ]:
def models_request():
    form_data = {
        'hardware': json.dumps(hardware),
        'bgo_dag': json.dumps(input_dag)
    }
    models_response = requests.post(url + '/models', data=form_data)
    return models_response.text

In [ ]:
# Dropdown and slider functionality
clear_output()

microbenchmarks = hardware['hosts'][0]['cpus']['benchmarks']
microbenchmark_name = list(microbenchmarks.keys())[0]
dropdown = Dropdown(options=microbenchmarks.keys(), description='Variable')
slider = FloatSlider(min=0, max=200, step=0.01, description='Value', value=microbenchmarks[microbenchmark_name])
response = None

def set_microbenchmark_name(variable):
    global microbenchmark_name
    microbenchmark_name = variable


def update_slider_value(x):
    global microbenchmark_name
    microbenchmark_name = x
    slider.value = microbenchmarks[x]
    return slider.value


def update_microbenchmark_value(value):
    global response
    hardware['hosts'][0]['cpus']['benchmarks'][microbenchmark_name] = value
    print(models_request())


dlink((dropdown, 'value'), (slider, 'value'), update_slider_value)
display(Markdown('### Change the value of certain microbenchmarks, and see how they impact the performance models'))
interact(set_microbenchmark_name, variable=dropdown)
interact(update_microbenchmark_value, value=slider);


### Step 4: Specify graph characteristics

The final step is to specify the graph characteristics of a specific graph for which you want to predict the execution time. This can be done by submitting a POST request to the API with the input DAG, hardware configuration, and graph properties. The API will return the input DAG annotated with the predicted execution times.

The graph properties are expressed in a simple JSON format of which an example is given below:

In [ ]:
graph_props = {
    "n": 15763,
    "m": 171206,
    "average_degree": 21,
    "directed": False,
    "weighted": False,
    "clustering_coefficient": 0.0132526,
    "triangle_count": 591156,
    "s": 1000
}

This can be useful when analyzing theoretical performance of an algorithm on non-existing graphs. However, when predicting performance for a specific graph, a graph file can be given, of which the properties are automatically extracted. Try getting analytical performance prediction results for a graph by picking one with the following cell.

In [ ]:
fc = FileChooser('data/beta_testing')
fc.filter_pattern = '*.mtx'
display(fc)

In [ ]:
graph_name = fc.selected
g = nx.from_scipy_sparse_array(sp.io.mmread(graph_name))

# Extract graph properties from file
n = len(g.nodes())
m = len(g.edges())
extracted_properties = {
    "n": n,
    "m": m,
    "directed": g.is_directed(),
    "weighted": nx.is_weighted(g),
    "diameter": nx.diameter(g),
    "clustering_coefficient": nx.average_clustering(g),
    "triangle_count": sum(nx.triangles(g).values()) // 3
}

print("Extracted graph properties:", extracted_properties)
graph_props = extracted_properties

def evaluate_request():
    form_data = {
        'hardware': json.dumps(hardware),
        'bgo_dag': json.dumps(input_dag),
        'graph_props': json.dumps(graph_props)
    }
    evaluate_response = requests.post(url + '/evaluate', data=form_data)
    return evaluate_response.text

def plot_result(target):
    data = json.loads(evaluate_request())
    plot_annotated_dag(data, "Analytical performance predictions", target)

display(Markdown('### Change the graph in the cell above, and see how it impacts the performance and energy predictions'))
interact(plot_result, target=ToggleButtons(options=['runtime', 'energy'], description='Prediction target'));

As you can see, a lot of the implementations do not have an analytical model yet, as indicated by the red labels. To still get performance predictions for these implementations, we resort to sampling based performance prediction.

#### Sampling based performance prediction
This sampling based performance prediction works best on larger graphs. Therefore, we will continue with the higgs_social_network.mtx graph. Since calculating the graph properties can take a while, the pre-calculated properties are already listed below.

In [ ]:
graph_name = "data/beta_testing/large/higgs_social_network.mtx"
graph_props = {
    "n": 456290,
    "m": 12508244,
    "average_degree": 27.41292599,
    "directed": False,
    "weighted": False,
    "diameter": 9,
    "clustering_coefficient": 0.1887,
    "triangle_count": 83023401,
    "s": 1000
}

Sampling is done by randomly selecting a certain fraction of the nodes while reading the graph. Execute the graph sampling by running the following cell.

In [ ]:
sampling_rate = 0.1
sampled_graph_name = 'sampled.mtx'
!./sampling/main {graph_name} {sampling_rate} {sampled_graph_name} && echo "Succesfully sampled the graph"

In [ ]:
# This cell contains some functions for running benchmarks on the BGO implementations
def run_benchmark(bgo, graph_file):
    result = [item for item in subprocess.check_output(['python3', './autobench/run_bench.py', bgo, '--data', f'G={graph_file}']).decode('utf-8').rstrip().split('\n') if item != ''][-2:]
    values = dict(zip(*map(lambda x: x.split(','), result)))

    return values['runtime_ns'], values['energy_joules']

def benchmark_workflow(workflow, graph_file):
    devices = ["CPU", "GPU"]

    # Benchmark all bgos given in the dag, by going into all subdirectories of ./bgo/dag/CPU and ./bgo/dag/GPU, and checking if ./bench exists. If it exists, call run_benchmark with the bgo path and given graph file
    results = []
    for bgo in workflow:
        bgo_name = bgo["name"]
        bgo_result = {"name": bgo_name, "performances": []}
        performance_entry = {"host": "H01", "runtime": {"CPU": {}, "GPU": {}}, "energy": {"CPU": {}, "GPU": {}}}

        base_dir = f"./bgo/{bgo_name}"
        for device in devices:
            device_dir = os.path.join(base_dir, device)
            if not os.path.isdir(device_dir):
                continue

            # Iterate over implementations inside each device directory
            for impl in os.listdir(device_dir):
                impl_path = os.path.join(device_dir, impl)
                bench_exe = os.path.join(impl_path, "bench")

                if os.path.isfile(bench_exe) and os.access(bench_exe, os.X_OK):
                    clear_output(wait=True)
                    display(f"Benchmarking {bgo_name} [{device}/{impl}]")
                    try:
                        runtime_ns, energy_joules = run_benchmark(impl_path, graph_file)
                        performance_entry["runtime"][device][impl] = float(runtime_ns)
                        performance_entry["energy"][device][impl] = float(energy_joules)
                    except subprocess.CalledProcessError as e:
                        print(f"Benchmark failed for {impl_path}: {e}")
                    except Exception as e:
                        print(f"Error while benchmarking {impl_path}: could not load graph")

        bgo_result["performances"].append(performance_entry)
        results.append(bgo_result)

    clear_output(wait=True)
    display(f"Benchmarking completed")

    return results

In [ ]:
# Benchmark the BGOs from the workflow on the sampled graph
sampled_graph_benchmark = benchmark_workflow(input_dag, sampled_graph_name)

The following cell shows the benchmarked runtimes and energy consumptions for the sampled graph. It is possible that some BGO implementations are not shown, if there is not enough RAM available. This is not an issue for this tutorial, so you can ignore this.

In [ ]:
def plot_result(data, target):
    plot_annotated_dag(data, "Benchmarked performance values on the sampled graph", target)

interact(lambda target: plot_result(sampled_graph_benchmark, target), target=ToggleButtons(options=['runtime', 'energy'],description='Prediction target'));

#### Automatic validation
To validate the predictions made by the analytical models and sampling approach, we benchmark the same implementations on the full graph. We then compare the predictions against these benchmarks in the cells below.

In [ ]:
# Benchmark the BGOs from the workflow on the full graph
full_graph_benchmark = benchmark_workflow(input_dag, graph_name)

##### Analytical model validation

In [ ]:
def evaluate_request():
    form_data = {
        'hardware': json.dumps(hardware),
        'bgo_dag': json.dumps(input_dag),
        'graph_props': json.dumps(graph_props)
    }
    evaluate_response = requests.post(url + '/evaluate', data=form_data)
    return evaluate_response.text

def validate_analytical(target):
    prediction = json.loads(evaluate_request())
    analytical_prediction_validation(prediction, full_graph_benchmark, target)

    
interact(validate_analytical, target=ToggleButtons(options=['runtime', 'energy'],description='Prediction target'));

##### Sampling based prediction validation

In [ ]:
def validate_sampling(data, target):
    sampling_prediction_validation(data, full_graph_benchmark, sampling_rate, target)

interact(lambda target: validate_sampling(sampled_graph_benchmark, target), target=ToggleButtons(options=['runtime', 'energy'],description='Prediction target'));

### Stop prediction server

In [ ]:
# Stop the server
server.terminate()